In [1]:
import pandas as pd
import os
import copy
import unittest
import string
import re

def read_file(trainFile):
    pwd = os.getcwd()
    os.chdir(os.path.dirname(trainFile))
    file1 = open(trainFile, 'r') 
    lines = file1.read().splitlines()
    return lines

In [2]:
lines = read_file("C:/Code/advent/2021_day23_input.txt")

In [3]:
lines_example1 = read_file("C:/Code/advent/2021_day23_example1.txt")
lines_example1

['#############',
 '#...........#',
 '###B#C#B#D###',
 '  #A#D#C#A#',
 '  #########']

In [4]:
def parse(input):
    result = {}
    row = 0
    for line in input:
        col = 0
        for c in line:
            point = (col,row)
            if c != '#' and c != ' ':
                result[point] = c
            col += 1
        row += 1
    return result

def print_field(input):
    x_n = 0
    y_n = 0
    for point in input:
        x_n = max(x_n, point[0])
        y_n = max(y_n, point[1])
    for y in range(y_n+1):
        line = ''
        for x in range(x_n+1):
            line += input.get((x,y),' ')
        print(line)
    print('')

print_field(parse(lines_example1))

            
 ...........
   B C B D  
   A D C A  



In [5]:
print_field(parse(lines))
print(parse(lines))

            
 ...........
   B B D D  
   C C A A  

{(1, 1): '.', (2, 1): '.', (3, 1): '.', (4, 1): '.', (5, 1): '.', (6, 1): '.', (7, 1): '.', (8, 1): '.', (9, 1): '.', (10, 1): '.', (11, 1): '.', (3, 2): 'B', (5, 2): 'B', (7, 2): 'D', (9, 2): 'D', (3, 3): 'C', (5, 3): 'C', (7, 3): 'A', (9, 3): 'A'}


In [6]:
def create_map_template(input):
    result = {}
    for key in input:
        result[key] = '.'
    return result

map_template = create_map_template(parse(lines))

def map_to_positions(input):
    result = {}
    for key in input:
        value = input[key]
        if value != '.':
            if value in result:
                result[value.lower()] = key
            else:
                result[value] = key
    return result

def positions_to_map(input):
    result = {}
    for key in map_template:
        result[key] = '.'
    for value in input:
        result[input[value]] = value
    return result

print(map_to_positions(parse(lines)))
print_field(positions_to_map(map_to_positions(parse(lines))))

{'B': (3, 2), 'b': (5, 2), 'D': (7, 2), 'd': (9, 2), 'C': (3, 3), 'c': (5, 3), 'A': (7, 3), 'a': (9, 3)}
            
 ...........
   B b D d  
   C c A a  



In [7]:
def energy_for_type(input):
    if input == 'a' or input == 'A':
        return 1
    if input == 'b' or input == 'B':
        return 10
    if input == 'c' or input == 'C':
        return 100
    if input == 'd' or input == 'D':
        return 1000

In [8]:
def create_route_next(start, end):
    if start[0] == end[0]:
        # move vertical
        if start[1] < end[1]:
            return (start[0],start[1]+1)
        elif start[1] > end[1]:            
            return (start[0],start[1]-1)
    elif start[1] == 1:
        # move horizontal
        if start[0] < end[0]:
            return (start[0]+1,start[1])
        elif start[0] > end[0]:
            return (start[0]-1,start[1])            
    else:
        # move in U
        return (start[0],start[1]-1)
    
    print(start,end)
    return 1 + 'should not reach'

def create_route(start, end):
    result = []
    next = start
    while next != end:
        next = create_route_next(next,end)
        result.append(next)
    return result

def get_valid_places_template(input):
    valid_places = copy.deepcopy(map_template)
    for i in range(0,4):
        valid_places.pop((3+2*i,1))
    return valid_places
print_field(get_valid_places_template(map_template))
valid_places = get_valid_places_template(map_template)

def create_all_routes():
    result = {}
    for start in valid_places:
        for end in valid_places:
            result[(start, end)] = create_route(start,end)
    return result

#print(create_route((3,3),(3,2)))
#print(create_route((3,3),(5,3)))
all_routes = create_all_routes()
print(len(all_routes))
print(all_routes[((3,3),(5,3))])

            
 .. . . . ..
   . . . .  
   . . . .  

225
[(3, 2), (3, 1), (4, 1), (5, 1), (5, 2), (5, 3)]


In [9]:
def is_valid_move(map, start, end):
    route = all_routes[(start,end)]
    for point in route:
        if map[point] != '.':
            return False
    return True

def get_pieces(map):
    result = {}
    for point in map:
        value = map[point] 
        if value != '.':
            result[point] = value
    return result

def is_solved(map, point):
    (x,y) = point
    if y == 1:
        return False
    piece = map[point]
    if piece == 'A' and x != 3:
        return False
    if piece == 'B' and x != 5:
        return False
    if piece == 'C' and x != 7:
        return False
    if piece == 'D' and x != 9:
        return False
    if point[1] == 2 and map[(x,y+1)] != piece:
        return False
    return True

def get_unsolved_pieces(map):
    pieces = get_pieces(map)
    result = []
    for point in pieces:
        if not is_solved(map, point):
            result.append(point)
    return result

print(get_unsolved_pieces(parse(lines)))
print(get_unsolved_pieces(parse(lines_example1)))

[(3, 2), (5, 2), (7, 2), (9, 2), (3, 3), (5, 3), (7, 3), (9, 3)]
[(3, 2), (5, 2), (7, 2), (9, 2), (5, 3), (9, 3)]


In [44]:
def sort_key(state):
    return -state[1]   

def select_state_with_lowest_cost(states):
    highest = 1000000
    index = None
    for i in range(len(states)):
        m, score, depth = states[i]
        if score < highest:
            index = i
            highest = score
    return index

def select_state_with_highest_depth(states):
    best_depth = -1
    best_cost = 1000000
    index = None
    for i in range(len(states)):
        m, score, depth = states[i]
        if depth > best_depth:
            index = i
            best_depth = depth
            best_cost = score
        elif depth == best_depth:
            if score < best_cost:
                index = i
                best_depth = depth
                best_cost = score
    return index

def get_target_x_for_animal(animal):
    if animal == 'A':
        return 3
    elif animal == 'B':
        return 5
    elif animal == 'C':
        return 7
    elif animal == 'D':
        return 9

def is_valid_destination(current_map, start,end, animal):
    if end[1] == 3:
        # going to final position bottom
        return get_target_x_for_animal(animal) == end[0]
    if end[1] == 2:
        # going to final position
        return current_map[(end[0],3)] == animal and get_target_x_for_animal(animal) == end[0]
    else:
        # going to parking position
        return start[1] > 1

def get_allowed_moves(current_map, piece):
    result = []
    for target in valid_places:
        if not is_valid_destination(current_map, piece, target, current_map[piece]):
            continue        
        if not is_valid_move(current_map, piece, target):
            continue        
        result.append(target)        
    return result

def move(current, piece, target):
    current_map, current_score, current_depth = current
    next_map = copy.deepcopy(current_map)    
    next_map[piece] = '.'
    next_map[target] = current_map[piece]
    cost = len(all_routes[(piece,target)]) * energy_for_type(current_map[piece])
    return (next_map, cost+current_score, current_depth+1)

def calculate_part_one(input):
    starting_state = (input, 0, 0)
    remaining_states = [starting_state]
    max_iterations = 40000000
    n = 0
    best = None
    while len(remaining_states) > 0 and n < max_iterations:
        n+=1        
        #select state to investigate
        index = len(remaining_states) - 1
                
        #if len(remaining_states) < 50:
            #index = select_state_with_lowest_cost(remaining_states)
        #else:
            #index = select_state_with_highest_depth(remaining_states)
        

        current = remaining_states.pop(index)         
        current_map, current_score, current_depth = current

        if n % 50000 == 0:
            min_index = select_state_with_lowest_cost(remaining_states)
            min_score = remaining_states[min_index][1]
            print('iteration', n, 'best', best[1] if best != None else None, 'min_score', min_score, 'score', current_score, 'depth', current_depth,len(remaining_states))

        if best != None:
            if best[1] <= current_score:
                continue

        if (not best == None) and current_score > best[1]:
            # everything else is more expensive
            break
        current_unsolved = get_unsolved_pieces(current_map)
        if len(current_unsolved) == 0:
            # done with this one
            if best == None:
                best = current
            elif current_score < best[1]:
                best = current
        else:
            for piece in current_unsolved:
                targets = get_allowed_moves(current_map, piece)
                new_states = []
                for t in targets:
                    next = move(current, piece, t)                    
                    new_states.append(next)                    
                    #print_field(next[0])
                    #print('score',next[1], 'depth',next[2])
                    #print('-----------------------------')
                new_states.sort(key=sort_key)
                for state in new_states:
                    remaining_states.append(state)

    print_field(best[0])

    return best[1]
    
calculate_part_one(parse(lines_example1))

iteration 50000 best 12539 min_score 20 score 12699 depth 10 58
iteration 100000 best 12539 min_score 20 score 10679 depth 9 58
iteration 150000 best 12521 min_score 20 score 9143 depth 11 60
iteration 200000 best 12521 min_score 20 score 11103 depth 10 64
iteration 250000 best 12521 min_score 20 score 9143 depth 9 55
iteration 300000 best 12521 min_score 20 score 10799 depth 11 55
iteration 350000 best 12521 min_score 20 score 12799 depth 12 52
iteration 400000 best 12521 min_score 20 score 12139 depth 10 48
iteration 450000 best 12521 min_score 20 score 18629 depth 10 39
iteration 500000 best 12521 min_score 20 score 5799 depth 7 46
iteration 550000 best 12521 min_score 20 score 5673 depth 7 35
iteration 600000 best 12521 min_score 20 score 11513 depth 9 60
iteration 650000 best 12521 min_score 20 score 12454 depth 7 50
iteration 700000 best 12521 min_score 20 score 12494 depth 9 47
iteration 750000 best 12521 min_score 20 score 8693 depth 8 49
iteration 800000 best 12521 min_score 2

KeyboardInterrupt: 

In [45]:
# 10431 too high
# 10411 ~ 3M

calculate_part_one(parse(lines))

iteration 50000 best 10647 min_score 20 score 10683 depth 15 82
iteration 100000 best 10647 min_score 20 score 10489 depth 15 78
iteration 150000 best 10647 min_score 20 score 10699 depth 14 82
iteration 200000 best 10647 min_score 20 score 10437 depth 13 78
iteration 250000 best 10647 min_score 20 score 7993 depth 12 67
iteration 300000 best 10647 min_score 20 score 10117 depth 12 67
iteration 350000 best 10647 min_score 20 score 10887 depth 15 66
iteration 400000 best 10647 min_score 20 score 11245 depth 15 67
iteration 450000 best 10647 min_score 20 score 10689 depth 15 65
iteration 500000 best 10647 min_score 20 score 10677 depth 13 65
iteration 550000 best 10647 min_score 20 score 10689 depth 16 72
iteration 600000 best 10647 min_score 20 score 10497 depth 14 66
iteration 650000 best 10647 min_score 20 score 10077 depth 13 66
iteration 700000 best 10647 min_score 20 score 10637 depth 14 72
iteration 750000 best 10647 min_score 20 score 9999 depth 13 63
iteration 800000 best 10647 

KeyboardInterrupt: 

In [12]:
calculate_part_two(parse(lines))

NameError: name 'calculate_part_two' is not defined